_Author: Dmitry Bagaev_

In [ ]:
using DrWatson

In [ ]:
@quickactivate "RxInferThesisExperiments"

In [ ]:
using RxInferThesisExperiments, Turing, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames, Logging

In [ ]:
Turing.setprogress!(false)

In [ ]:
const bfolder_nuts = datadir("lds", "turing", "nuts")

In [ ]:
# Pregenerate paths for benchmark data
mkpath(bfolder_nuts);

In [ ]:
# Include the model specification
include(srcdir("models", "turing", "rotating.jl"));

In [ ]:
function run_benchmark_nuts(params)
    return with_logger(NullLogger()) do
        @unpack T, d, nsamples, seed = params

        environment = RotatingTracking(d; rng = StableRNG(seed))

        states, observations = rand(StableRNG(seed), environment, T);
        model    = rotating(observations, environment.A, environment.B, environment.P, environment.Q) 
        result   = run_inference_sampling(model; nsamples = nsamples, rng = StableRNG(seed));
        e_states = extract_posteriors(T, environment.d, result)
        amse     = compute_amse(states, e_states)

        benchmark_modelcreation = @benchmark rotating($observations, $(environment.A), $(environment.B), $(environment.P), $(environment.Q))


        benchmark_inference = @benchmark run_inference_sampling(model; nsamples = $nsamples, rng = StableRNG($seed)) setup=begin
            model = rotating($observations, $(environment.A), $(environment.B), $(environment.P), $(environment.Q))
            states, observations = rand(StableRNG($seed), $environment, $T);
        end

        emse = compute_emse(seed) do _seed
            local environment = RotatingTracking(d; rng = StableRNG(_seed))
            local states, observations = rand(StableRNG(_seed), environment, T);
            local model    = rotating(observations, environment.A, environment.B, environment.P, environment.Q) 
            local result   = run_inference_sampling(model; nsamples = nsamples, rng = StableRNG(_seed))
            local e_states = extract_posteriors(T, environment.d, result)
            return compute_amse(states, e_states)
        end

        output = @strdict T d nsamples seed states e_states observations amse emse benchmark_modelcreation benchmark_inference

        return output
    end
end

In [ ]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params_nuts = dict_list(Dict(
    "T"    => [ 10, 20, 30, 50, 100, 200 ],
    "d"    => [ 2, 3, 4 ],
    "nsamples" => [ 100, 200, 300 ],
    "seed" => [ 42 ]
));

In [ ]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks_nuts = map(benchmark_params_nuts) do params
    result, _ = produce_or_load(run_benchmark_nuts, bfolder_nuts, params; tag = false, force = false)
    return result
end;

In [ ]:
sort(prepare_benchmarks_table(bfolder_nuts), [ :T ])

# Versions

In [ ]:
versioninfo()

In [ ]:
] status